In [ ]:
import requests
import time

def get_artist_info(artist_name, endpoint_url="https://query.wikidata.org/sparql", retries=3, delay=1):
    # SPARQL query to fetch the birth place, birth date, death date, and place of death
    query = '''
    SELECT ?artist ?artistLabel ?placeOfBirthLabel ?dateOfBirth ?dateOfDeath ?placeOfDeathLabel WHERE {
      ?artist ?label "%s"@en.
      ?artist wdt:P19 ?placeOfBirth.
      ?artist wdt:P569 ?dateOfBirth.
      ?artist wdt:P570 ?dateOfDeath.
      ?artist wdt:P20 ?placeOfDeath.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    LIMIT 1
    ''' % artist_name.replace('"', '\"')

    # Attempt the request up to 'retries' times
    for attempt in range(retries):
        response = requests.get(endpoint_url, params={'query': query, 'format': 'json'})
        
        if response.status_code == 200:
            data = response.json()
            results = data.get('results', {}).get('bindings', [])
            if results:
                artist_info = {
                    'name': artist_name,
                    'birth_place': results[0].get('placeOfBirthLabel', {}).get('value', None),
                    'birth_date': results[0].get('dateOfBirth', {}).get('value', None),
                    'death_date': results[0].get('dateOfDeath', {}).get('value', None),
                    'death_place': results[0].get('placeOfDeathLabel', {}).get('value', None),
                }
                return artist_info
            break  # Break out of the loop if successful
        else:
            print(f"Error fetching data for {artist_name}, status code: {response.status_code}. Attempt {attempt + 1} of {retries}.")
            if response.status_code in [429, 500, 502, 503, 504]:
                time.sleep(delay * (attempt + 1))  # Exponential back-off
            else:
                break  # Break on non-retryable error

    return None

https://www.wikidata.org/wiki/Q5582
https://www.wikidata.org/wiki/Property:P580